In [1]:
from PyQt5.QtWidgets import QMainWindow, QApplication, QLabel, QFileDialog, QMessageBox, QColorDialog, QVBoxLayout, QPushButton, QWidget, QHBoxLayout, QSlider, QComboBox, QCheckBox
from PyQt5.QtGui import QPixmap, QImage, QColor
from PyQt5.QtCore import QDir, Qt, QTimer
from PyQt5 import QtWidgets, QtGui, QtCore
import cv2
import sys
import numpy as np

from matplotlib import pyplot as plt

class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()

        self.blackboard = None
        self.frame = None
        
        self.noise = 500
        
        self.color = (255, 255, 255)
        self.draw = False
        self.switch = False
        
        self.lowH = 0
        self.highH = 0
        self.lowS = 0
        self.highS = 0
        self.lowV = 0
        self.highV = 0
        
        self.x1 = 0
        self.y1 = 0
        
        self.cap = cv2.VideoCapture(0)
        self.cap = cv2.VideoCapture(0)
        self.cap.set(3,1440)
        self.cap.set(4,600)
        
        self.labelVideo = QLabel(self)
        self.labelVideo.setGeometry(10, 10, 1400, 600)

        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_video)
        self.timer.start(30)
        
        self.create_ui()

    def create_ui(self):
        # Main Window
        self.setWindowTitle('T08')
        self.setGeometry(100, 100, 1440, 900)
        
        # LowH
        self.sliderLowH = QSlider(self)
        self.sliderLowH.setOrientation(Qt.Horizontal)
        self.sliderLowH.setObjectName("SliderLowH")
        self.sliderLowH.setMaximum(179)
        self.sliderLowH.setGeometry(50, 740, 300, 30)
        self.sliderLowH.valueChanged.connect(self.lowH_changeValue)
        
        # HighH
        self.sliderHighH = QSlider(self)
        self.sliderHighH.setOrientation(Qt.Horizontal)
        self.sliderHighH.setObjectName("SliderHighH")
        self.sliderHighH.setMaximum(179)
        self.sliderHighH.setGeometry(50, 780, 300, 30)
        self.sliderHighH.valueChanged.connect(self.highH_changeValue)
        
        self.labelLowH = QLabel('LowH: ', self)
        self.labelLowH.setGeometry(140, 820, 80, 30)

        self.labelHighH = QLabel('HighH: ', self)
        self.labelHighH.setGeometry(240, 820, 80, 30)
        
        # LowS
        self.sliderLowS = QSlider(self)
        self.sliderLowS.setOrientation(Qt.Horizontal)
        self.sliderLowS.setObjectName("SliderLowS")
        self.sliderLowS.setMaximum(255)
        self.sliderLowS.setGeometry(400, 740, 300, 30)
        self.sliderLowS.valueChanged.connect(self.lowS_changeValue)
        
        # HighS
        self.sliderHighS = QSlider(self)
        self.sliderHighS.setOrientation(Qt.Horizontal)
        self.sliderHighS.setObjectName("SliderHighS")
        self.sliderHighS.setMaximum(255)
        self.sliderHighS.setGeometry(400, 780, 300, 30)
        self.sliderHighS.valueChanged.connect(self.highS_changeValue)
        
        self.labelLowS = QLabel('LowS: ', self)
        self.labelLowS.setGeometry(490, 820, 80, 30)
        
        self.labelHighS = QLabel('HighS: ', self)
        self.labelHighS.setGeometry(590, 820, 80, 30)
        
        # LowV
        self.sliderLowV = QSlider(self)
        self.sliderLowV.setOrientation(Qt.Horizontal)
        self.sliderLowV.setObjectName("SliderLowV")
        self.sliderLowV.setMaximum(255)
        self.sliderLowV.setGeometry(750, 740, 300, 30)
        self.sliderLowV.valueChanged.connect(self.lowV_changeValue)
        
        # HighV
        self.sliderHighV = QSlider(self)
        self.sliderHighV.setOrientation(Qt.Horizontal)
        self.sliderHighV.setObjectName("SliderHighV")
        self.sliderHighV.setMaximum(255)
        self.sliderHighV.setGeometry(750, 780, 300, 30)
        self.sliderHighV.valueChanged.connect(self.highV_changeValue)
        
        self.labelLowV = QLabel('LowV: ', self)
        self.labelLowV.setGeometry(850, 820, 80, 30)
        
        self.labelHighV = QLabel('HighV: ', self)
        self.labelHighV.setGeometry(950, 820, 80, 30)
        
        # Pick Color
        self.buttonPickColor = QtWidgets.QPushButton("Alege culoarea", self)
        self.buttonPickColor.setObjectName("ButtonPickColor")
        self.buttonPickColor.setGeometry(1100, 740, 150, 30)
        self.buttonPickColor.clicked.connect(self.pickColor)
        
        # Choosen color
        self.labelColor = QLabel('', self)
        self.labelColor.setGeometry(1260, 740, 30, 30)
        
        self.checkboxDraw = QCheckBox('draw', self)
        self.checkboxDraw.setObjectName("CheckboxDraw")
        self.checkboxDraw.setGeometry(1300, 740, 150, 30)
        self.checkboxDraw.stateChanged.connect(self.draw_checkbox)
        
        self.checkboxSwitch = QCheckBox('switch', self)
        self.checkboxSwitch.setObjectName("CheckboxSwitch")
        self.checkboxSwitch.setGeometry(1300, 780, 150, 30)
        self.checkboxSwitch.stateChanged.connect(self.switch_checkbox)
         
        self.buttonClear = QtWidgets.QPushButton("Sterge desenul", self)
        self.buttonClear.setObjectName("ButtonClear")
        self.buttonClear.setGeometry(1100, 780, 150, 30)
        self.buttonClear.clicked.connect(self.clearBlackboard)
        
        self.buttonExit = QPushButton('Exit', self)
        self.buttonExit.setObjectName("ButtonExit")
        self.buttonExit.setGeometry(1100, 820, 150, 30)
        self.buttonExit.clicked.connect(self.exit)
        
    def pickColor(self):
        new_color = QColorDialog.getColor(QColor(*self.color), self, 'Pick a Color')
        if new_color.isValid():
            self.color = (new_color.blue(), new_color.green(), new_color.red())
            style = f'background-color: {new_color.name()}; border: 1px solid black;'
            self.labelColor.setStyleSheet(style)

    def draw_checkbox(self, state):
        if state == Qt.Checked:
            self.draw = True
        else:
            self.draw = False
            
    def switch_checkbox(self, state):
        if state == Qt.Checked:
            self.switch = True
        else:
            self.switch = False

    def lowH_changeValue(self):
        self.lowH = self.sliderLowH.value()
        new_text = f'LowH: {self.lowH}'
        self.labelLowH.setText(new_text)

    def highH_changeValue(self):
        self.highH = self.sliderHighH.value()
        new_text = f'HighH: {self.highH}'
        self.labelHighH.setText(new_text)

    def lowS_changeValue(self):
        self.lowS = self.sliderLowS.value()
        new_text = f'LowS: {self.lowS}'
        self.labelLowS.setText(new_text)
        
    def highS_changeValue(self):
        self.highS = self.sliderHighS.value()
        new_text = f'HighS: {self.highS}'
        self.labelHighS.setText(new_text)
        
    def lowV_changeValue(self):
        self.lowV = self.sliderLowV.value()
        new_text = f'LowV: {self.lowV}'
        self.labelLowV.setText(new_text)
        
    def highV_changeValue(self):
        self.highV = self.sliderHighV.value()
        new_text = f'HighV: {self.highV}'
        self.labelHighV.setText(new_text)

    def update_video(self):
        ret, frame = self.cap.read()
        frame = cv2.flip( frame, 1 )
        if ret:
            webcam_width = int(self.cap.get(3))
            webcam_height = int(self.cap.get(4))

            if self.blackboard is None:
                self.blackboard = np.zeros_like(frame)

            if self.switch == False:
                formatHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                masca = cv2.inRange(formatHSV, np.array([self.lowH, self.lowS, self.lowV]), np.array([self.highH, self.highS, self.highV]))
                kernel = np.ones((3, 3), np.uint8)
                dilatata = cv2.dilate(masca, kernel, iterations=2)
                erodata = cv2.erode(dilatata, kernel, iterations=2)

                processed_image = cv2.cvtColor(erodata, cv2.COLOR_GRAY2RGB)

                processed_image = cv2.resize(processed_image, (webcam_width, webcam_height))

                q_image = QImage(processed_image.data, webcam_width, webcam_height, 3 * webcam_width, QImage.Format_RGB888).rgbSwapped()
            else:
                formatHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                masca = cv2.inRange(formatHSV, np.array([self.lowH, self.lowS, self.lowV]), np.array([self.highH, self.highS, self.highV]))
                kernel = np.ones((3, 3), np.uint8)
                dilatata = cv2.dilate(masca, kernel, iterations=2)
                erodata = cv2.erode(dilatata, kernel, iterations=2)

                contours, hierarchy = cv2.findContours(erodata, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > self.noise:
                    c = max(contours, key=cv2.contourArea)    
                    x2, y2, w, h = cv2.boundingRect(c)
                    if self.x1 == 0 and self.y1 == 0:
                        self.x1, self.y1 = x2, y2     
                    elif self.draw == True:
                        self.blackboard = cv2.line(self.blackboard, (self.x1, self.y1), (x2, y2), self.color, 4) 
                    self.x1, self.y1 = x2, y2
                else:
                    self.x1, self.y1 = 0, 0

                self.blackboard = cv2.resize(self.blackboard, (webcam_width, webcam_height))

                frame = cv2.resize(frame, (webcam_width, webcam_height))
                frame = cv2.add(frame, self.blackboard)

                stacked = np.hstack((self.blackboard, frame))
                stacked = cv2.resize(stacked, (webcam_width, webcam_height))

                q_image = QImage(stacked.data, webcam_width, webcam_height, 3 * webcam_width, QImage.Format_RGB888).rgbSwapped()

            pixmap = QPixmap.fromImage(q_image)
            self.labelVideo.setPixmap(pixmap)

    def clearBlackboard(self):
        self.blackboard = None
        
    def exit(self):
        self.cap.release()
        self.timer.stop()

        cv2.destroyAllWindows()
        cv2.waitKey(1)
        self.close()

if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\crist\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
